<a href="https://colab.research.google.com/github/ramalamadingdong/yolo-rb3gen2-trainer/blob/main/AI_Hub_Quanitization_RB3Gen2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This file should be ran Locally!

## RB3 Gen 2 (Proxy)
I recommend the following be run on your personal machine but you can do this on the Colab too however the last step takes a long time to complete.

In [ ]:
AIHUB_API_KEY = "YOUR_API_TOKEN"
PATH_TO_WEIGHTS = "PATH_TO_WEIGHTS"
# This installs Python package
!pip install qai-hub-models[yolov8-det-quantized]
!pip install fiftyone
!pip install ultralytics
# sets up AI Hub enviroment
!qai-hub configure --api_token AIHUB_API_KEY

# Converts the model to be ran on RB3Gen2
!python -m qai_hub_models.models.yolov8_det.export --quantize w8a8 --device="RB3 Gen 2 (Proxy)" --ckpt-name PATH_TO_WEIGHTS --device-os linux --target-runtime tflite

To retrieve the quanatization parameters from the model the easiest way is the following:

In [ ]:
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="PATH_TO_FILE_FROM_AIHUB")
interpreter.allocate_tensors()

# Get output tensor details
output_details = interpreter.get_output_details()

q_scales = []
q_zero_points = []

for output in output_details:
    params = output['quantization_parameters']

    # Check if the tensor is quantized
    if output['quantization_parameters']['quantized_dimension'] == 0:  # Per-tensor quantization
        scale = params['scales'][0] if params['scales'].size > 0 else 1.0
        zero_point = params['zero_points'][0] if params['zero_points'].size > 0 else 0.0
    else:
        # Handle per-channel quantization (unlikely for YOLOv8)
        scale = 1.0
        zero_point = 0.0

    q_scales.append(scale)
    q_zero_points.append(float(zero_point))

# Format the constants
print(f'YOLOv8,q-offsets=<{", ".join(map(str, q_zero_points))}>, q-scales=<{", ".join(map(str, q_scales))}>')

To run on the RB3Gen2 with the NPU all you need is the following command replace the output from the above script with the constants on the cell below

In [ ]:
!gst-launch-1.0 -e --gst-debug=2 \
qtiqmmfsrc name=camsrc ! video/x-raw\(memory:GBM\),format=NV12,width=1280,height=720,framerate=30/1,compression=ubwc ! queue ! tee name=split \
split. ! queue ! qtivcomposer name=mixer ! queue ! waylandsink fullscreen=true \
split. ! queue ! qtimlvconverter ! queue ! qtimltflite delegate=external external-delegate-path=libQnnTFLiteDelegate.so \
external-delegate-options="QNNExternalDelegate,backend_type=htp;" model=/opt/yolov8_det_quantized.tflite ! queue ! \
qtimlvdetection threshold=50.0 results=10 module=yolov8 labels=/opt/coco_labels.txt constants="YOLOv8,q-offsets=<21.0, 0.0, 0.0>,q-scales=<3.093529462814331, 0.00390625, 1.0>;" ! \
video/x-raw,format=BGRA,width=640,height=360 ! queue ! mixer.
